# Graph Generation
This notebook outlines how to use the Urbanity python package to generate heterogeneous city graph objects. 

### Import Libraries

In [13]:
import random
import torch
import re
import ast

import urbanity as urb
from urbanity.utils import most_frequent
from urbanity.geom import project_gdf
from urbanity.utils import (get_building_to_building_edges,
                            get_buildings_in_plot_edges,
                             get_plot_to_plot_edges, 
                             get_building_to_street_edges,
                               get_edges_along_plot,
                               get_edge_nodes,
                               get_intersection_to_street_edges)

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from urbanity.population import raster2gdf

print(f'CUDA is available: {torch.cuda.is_available()}')

CUDA is available: False


### Generate Urban Graph with Urbanity

In [4]:
# Load demo area in Seattle
seattle_squire = gpd.read_parquet('../data/Seattle_Squire_Park.parquet')

In [5]:
# Create a Map object and specify shapefile
m1 = urb.Map('United States')
m1.polygon_bounds = seattle_squire

In [ ]:
seattle_buildings = m1.get_building_nodes('Seattle')

In [10]:
building_urban_analytics_indicators = [
 'bid',
 'building_area',
 'building_centroid',
 'building_perimeter',
 '3-nn-idx',
 '3-dist',
 '3-nn-threshold',
 '5-nn-idx',
 '5-dist',
 '5-nn-threshold',
 '8-nn-idx',
 '8-dist',
 '8-nn-threshold',
 '10-nn-idx',
 '10-dist',
 '10-nn-threshold',
 'building_circ_compact',
 'building_convexity',
 'building_corners',
 'building_elongation',
 'building_orientation',
 'building_longest_axis_length',
 'building_eri',
 'building_fractaldim',
 'building_rectangularity',
 'building_square_compactness',
 'building_shape_idx',
 'building_squareness',
 'building_complexity',
 '3-nn-idx_building_area_mean',
 '3-nn-idx_building_area_stdev',
 '5-nn-idx_building_area_mean',
 '5-nn-idx_building_area_stdev',
 '8-nn-idx_building_area_mean',
 '8-nn-idx_building_area_stdev',
 '10-nn-idx_building_area_mean',
 '10-nn-idx_building_area_stdev',
 '3-nn-idx_building_perimeter_mean',
 '3-nn-idx_building_perimeter_stdev',
 '5-nn-idx_building_perimeter_mean',
 '5-nn-idx_building_perimeter_stdev',
 '8-nn-idx_building_perimeter_mean',
 '8-nn-idx_building_perimeter_stdev',
 '10-nn-idx_building_perimeter_mean',
 '10-nn-idx_building_perimeter_stdev',
 '3-nn-idx_building_circ_compact_mean',
 '3-nn-idx_building_circ_compact_stdev',
 '5-nn-idx_building_circ_compact_mean',
 '5-nn-idx_building_circ_compact_stdev',
 '8-nn-idx_building_circ_compact_mean',
 '8-nn-idx_building_circ_compact_stdev',
 '10-nn-idx_building_circ_compact_mean',
 '10-nn-idx_building_circ_compact_stdev',
 '3-nn-idx_building_convexity_mean',
 '3-nn-idx_building_convexity_stdev',
 '5-nn-idx_building_convexity_mean',
 '5-nn-idx_building_convexity_stdev',
 '8-nn-idx_building_convexity_mean',
 '8-nn-idx_building_convexity_stdev',
 '10-nn-idx_building_convexity_mean',
 '10-nn-idx_building_convexity_stdev',
 '3-nn-idx_building_corners_mean',
 '3-nn-idx_building_corners_stdev',
 '5-nn-idx_building_corners_mean',
 '5-nn-idx_building_corners_stdev',
 '8-nn-idx_building_corners_mean',
 '8-nn-idx_building_corners_stdev',
 '10-nn-idx_building_corners_mean',
 '10-nn-idx_building_corners_stdev',
 '3-nn-idx_building_elongation_mean',
 '3-nn-idx_building_elongation_stdev',
 '5-nn-idx_building_elongation_mean',
 '5-nn-idx_building_elongation_stdev',
 '8-nn-idx_building_elongation_mean',
 '8-nn-idx_building_elongation_stdev',
 '10-nn-idx_building_elongation_mean',
 '10-nn-idx_building_elongation_stdev',
 '3-nn-idx_building_orientation_mean',
 '3-nn-idx_building_orientation_stdev',
 '5-nn-idx_building_orientation_mean',
 '5-nn-idx_building_orientation_stdev',
 '8-nn-idx_building_orientation_mean',
 '8-nn-idx_building_orientation_stdev',
 '10-nn-idx_building_orientation_mean',
 '10-nn-idx_building_orientation_stdev',
 '3-nn-idx_building_longest_axis_length_mean',
 '3-nn-idx_building_longest_axis_length_stdev',
 '5-nn-idx_building_longest_axis_length_mean',
 '5-nn-idx_building_longest_axis_length_stdev',
 '8-nn-idx_building_longest_axis_length_mean',
 '8-nn-idx_building_longest_axis_length_stdev',
 '10-nn-idx_building_longest_axis_length_mean',
 '10-nn-idx_building_longest_axis_length_stdev',
 '3-nn-idx_building_eri_mean',
 '3-nn-idx_building_eri_stdev',
 '5-nn-idx_building_eri_mean',
 '5-nn-idx_building_eri_stdev',
 '8-nn-idx_building_eri_mean',
 '8-nn-idx_building_eri_stdev',
 '10-nn-idx_building_eri_mean',
 '10-nn-idx_building_eri_stdev',
 '3-nn-idx_building_fractaldim_mean',
 '3-nn-idx_building_fractaldim_stdev',
 '5-nn-idx_building_fractaldim_mean',
 '5-nn-idx_building_fractaldim_stdev',
 '8-nn-idx_building_fractaldim_mean',
 '8-nn-idx_building_fractaldim_stdev',
 '10-nn-idx_building_fractaldim_mean',
 '10-nn-idx_building_fractaldim_stdev',
 '3-nn-idx_building_rectangularity_mean',
 '3-nn-idx_building_rectangularity_stdev',
 '5-nn-idx_building_rectangularity_mean',
 '5-nn-idx_building_rectangularity_stdev',
 '8-nn-idx_building_rectangularity_mean',
 '8-nn-idx_building_rectangularity_stdev',
 '10-nn-idx_building_rectangularity_mean',
 '10-nn-idx_building_rectangularity_stdev',
 '3-nn-idx_building_squareness_mean',
 '3-nn-idx_building_squareness_stdev',
 '5-nn-idx_building_squareness_mean',
 '5-nn-idx_building_squareness_stdev',
 '8-nn-idx_building_squareness_mean',
 '8-nn-idx_building_squareness_stdev',
 '10-nn-idx_building_squareness_mean',
 '10-nn-idx_building_squareness_stdev',
 '3-nn-idx_building_square_compactness_mean',
 '3-nn-idx_building_square_compactness_stdev',
 '5-nn-idx_building_square_compactness_mean',
 '5-nn-idx_building_square_compactness_stdev',
 '8-nn-idx_building_square_compactness_mean',
 '8-nn-idx_building_square_compactness_stdev',
 '10-nn-idx_building_square_compactness_mean',
 '10-nn-idx_building_square_compactness_stdev',
 '3-nn-idx_building_shape_idx_mean',
 '3-nn-idx_building_shape_idx_stdev',
 '5-nn-idx_building_shape_idx_mean',
 '5-nn-idx_building_shape_idx_stdev',
 '8-nn-idx_building_shape_idx_mean',
 '8-nn-idx_building_shape_idx_stdev',
 '10-nn-idx_building_shape_idx_mean',
 '10-nn-idx_building_shape_idx_stdev',
 '3-nn-idx_building_complexity_mean',
 '3-nn-idx_building_complexity_stdev',
 '5-nn-idx_building_complexity_mean',
 '5-nn-idx_building_complexity_stdev',
 '8-nn-idx_building_complexity_mean',
 '8-nn-idx_building_complexity_stdev',
 '10-nn-idx_building_complexity_mean',
 '10-nn-idx_building_complexity_stdev',
 'geometry']

# Subset building indicators
seattle_buildings = seattle_buildings[building_urban_analytics_indicators]

# Project to global coordinates
seattle_buildings = seattle_buildings.to_crs('epsg:4326')

In [ ]:
# Display buildings
seattle_buildings.plot()

### Process Building Dataframe

In [13]:
# Save building features
seattle_buildings['3-nn-idx'] = seattle_buildings['3-nn-idx'].apply(lambda x: str(x))
seattle_buildings['5-nn-idx'] = seattle_buildings['5-nn-idx'].apply(lambda x: str(x))
seattle_buildings['8-nn-idx'] = seattle_buildings['8-nn-idx'].apply(lambda x: str(x))
seattle_buildings['10-nn-idx'] = seattle_buildings['10-nn-idx'].apply(lambda x: str(x))
seattle_buildings['3-dist'] = seattle_buildings['3-dist'].apply(lambda x: str(x))
seattle_buildings['5-dist'] = seattle_buildings['5-dist'].apply(lambda x: str(x))
seattle_buildings['8-dist'] = seattle_buildings['8-dist'].apply(lambda x: str(x))
seattle_buildings['10-dist'] = seattle_buildings['10-dist'].apply(lambda x: str(x))
seattle_buildings['3-nn-threshold'] = seattle_buildings['3-nn-threshold'].apply(lambda x: str(x))
seattle_buildings['5-nn-threshold'] = seattle_buildings['5-nn-threshold'].apply(lambda x: str(x))
seattle_buildings['8-nn-threshold'] = seattle_buildings['8-nn-threshold'].apply(lambda x: str(x))
seattle_buildings['10-nn-threshold'] = seattle_buildings['10-nn-threshold'].apply(lambda x: str(x))
seattle_buildings.drop('building_centroid', axis='columns', inplace=True)

In [15]:
# Remove white space
seattle_buildings['3-nn-idx'] = seattle_buildings['3-nn-idx'].apply(lambda s: list(ast.literal_eval(re.sub("\s+", r",", s[1:-1].lstrip()))))

In [19]:
# Get building edges and save adjacency
building_to_building_edges, _  = get_building_to_building_edges(seattle_buildings, '3-nn-idx')
np.savetxt('../data/demo_building_to_building_edges.txt', building_to_building_edges, fmt='%d')

In [55]:
# Save building nodes
seattle_buildings.to_parquet('../data/demo_seattle_buildings.parquet')

### Get urban plots

In [ ]:
urban_plots = m1.get_urban_plot_nodes('Seattle')

In [ ]:
# Show urban plots
urban_plots.plot(column='plot_id')

### Manually add local climate zone

In [ ]:
lcz_gdf = raster2gdf('../seattle_files/LCZ_raster/Global_LCZ.tif', boundary=seattle_squire, zoom=True)
lcz_gdf.plot(column='value')

In [42]:
# Assign LCZ label to plot based on mode count
out = lcz_gdf.overlay(urban_plots)
mode_series = out.groupby('plot_id')['value'].aggregate(lambda x: most_frequent(list(x)))
mode_series.name = 'plot_lcz'
urban_plots = urban_plots.merge(mode_series, on='plot_id', how='left')

urban_plots['plot_id'] = urban_plots.index

### Get building in plot edges, plot to plot edges, and building to street edges

In [49]:
# Obtain plot to street edges
plot_to_street_edges, _ = get_edges_along_plot(urban_plots, adj_column = 'edge_ids')
np.savetxt('../data/demo_plot_to_street_edges.txt', plot_to_street_edges, fmt='%d')

# Obtain building to plot edges
building_to_plot_edges, _ = get_buildings_in_plot_edges(urban_plots, adj_column = 'building_ids')
np.savetxt('../data/demo_building_to_plot_edges.txt', building_to_plot_edges, fmt='%d')

# Obtain plot to plot edges
plot_to_plot_edges, _ = get_plot_to_plot_edges(urban_plots)
np.savetxt('../data/demo_plot_to_plot_edges.txt', plot_to_plot_edges, fmt='%d')

In [54]:
urban_plots = urban_plots.drop(columns = ['edge_ids', 'building_ids'], axis=1)
urban_plots.to_parquet('../data/demo_seattle_plots.parquet')

### Get building to street edges

In [3]:
network_graph, intersections, streets = m1.get_street_network('Seattle', svi_attr=True, edge_attr=True)

Network data found, skipping re-computation
Topologic/metric attributes computed. Time taken: 1.
SVI data found, skipping re-computation.
SVI attributes computed. Time taken: 2.
Total elapsed time --- 2 seconds ---


In [6]:
# Save street and intersection nodes
streets.to_parquet('../data/demo_seattle_streets.parquet')
intersections.to_parquet('../data/demo_seattle_intersections.parquet')

In [10]:
buildings = gpd.read_parquet('../data/demo_seattle_buildings.parquet')

In [19]:
# For each building (centroid), we want the nearest street representation. 
from urbanity.utils import get_building_to_street_edges
street_to_building_edges, _ = get_building_to_street_edges(streets,buildings)
np.savetxt('../data/demo_street_to_building_edges.txt', street_to_building_edges, fmt='%d')

### Get street to intersection edges

In [22]:
intersection_to_street_edges, _ = get_intersection_to_street_edges(intersections, streets)

np.savetxt('../data/demo_intersection_to_street_edges.txt', intersection_to_street_edges, fmt='%d')

### Link Building Energy Data

In [23]:
# Load processed seattle building energy data (https://data.seattle.gov/Built-Environment/2022-Building-Energy-Benchmarking/5sxi-iyiy/about_data)
building_energy_df = pd.read_csv('../energy_data/building_energy_labels.csv')

In [32]:
# Find buildings with labels
buildings_with_labels = buildings[buildings['bid'].isin(building_energy_df['bid'].values)]

### Create Open Data Train, Val, and Test Set

In [39]:
# Set random seed
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

# Buildings with labels
building_indices = buildings_with_labels.index

# Random shuffle sequences
np.random.shuffle(np.array(building_indices))

# Split entries into train, val, test
train_idx, val_idx, test_idx = building_indices[:int(0.7*len(building_indices))], \
                               building_indices[int(0.7*len(building_indices)):int(0.85*len(building_indices))], \
                               building_indices[int(0.85*len(building_indices)):]

In [42]:
index_to_bid = {k:v for k,v in zip(buildings_with_labels.index, buildings_with_labels['bid'])}

In [43]:
# Obtain bids array
train_bids = [index_to_bid[i] for i in train_idx]
val_bids = [index_to_bid[i] for i in val_idx]
test_bids = [index_to_bid[i] for i in test_idx]

In [45]:
np.savetxt('../data/demo_train_idx', train_idx, fmt='%s')
np.savetxt('../data/demo_val_idx', val_idx, fmt='%s')
np.savetxt('../data/demo_test_idx', test_idx, fmt='%s')
np.savetxt('../data/demo_train_bids', train_bids, fmt='%s')
np.savetxt('../data/demo_val_bids', val_bids, fmt='%s')
np.savetxt('../data/demo_test_bids', test_bids, fmt='%s')